In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString
from rtree import index
from GISio import shp2df, df2shp, get_values_at_points
from preproc import renumber_segments

In [2]:
newlines = '../Examples/data/added_lines.shp'
sfrlines = '../Examples/data/SFRlines.shp'
#dem = 'D:/ATLData/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'
dem = '/Users/aleaf/Documents/BR/BadRiver/grid/dem/dem_utm_ft/w001001.adf'

In [3]:
nl = shp2df(newlines)
sfr = shp2df(sfrlines)
tol = 100
nsegstart = sfr.segment.astype(int).max() + 1


reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)

reading ../Examples/data/SFRlines.shp...
--> building dataframe... (may take a while for large shapefiles)


In [4]:
r = renumber_segments(sfr.segment.values, sfr.outseg.values)

enforcing best segment numbering...


In [5]:
r

{0: 0,
 181: 1,
 197: 7,
 198: 6,
 199: 5,
 200: 4,
 201: 3,
 202: 2,
 217: 11,
 218: 10,
 219: 9,
 220: 8,
 236: 15,
 237: 14,
 238: 13,
 239: 12,
 259: 20,
 260: 19,
 284: 25,
 285: 24,
 292: 18,
 293: 17,
 294: 16,
 310: 36,
 311: 35,
 318: 94,
 319: 61,
 320: 60,
 322: 23,
 323: 22,
 324: 21,
 343: 44,
 344: 43,
 348: 0,
 349: 71,
 350: 70,
 351: 34,
 352: 33,
 353: 32,
 354: 31,
 355: 30,
 356: 29,
 357: 28,
 358: 27,
 359: 26,
 372: 50,
 373: 49,
 378: 80,
 379: 79,
 380: 42,
 381: 41,
 382: 40,
 383: 39,
 384: 38,
 385: 37,
 397: 54,
 398: 53,
 402: 93,
 403: 48,
 404: 47,
 405: 46,
 406: 45,
 425: 59,
 426: 58,
 428: 0,
 429: 52,
 430: 51,
 447: 69,
 448: 68,
 449: 67,
 450: 66,
 455: 57,
 456: 56,
 457: 55,
 474: 92,
 475: 91,
 476: 78,
 477: 77,
 480: 65,
 481: 64,
 482: 90,
 483: 89,
 498: 0,
 499: 88,
 501: 76,
 502: 75,
 503: 74,
 504: 0,
 521: 0,
 523: 87,
 524: 86,
 538: 108,
 550: 0,
 563: 107,
 571: 106,
 575: 63,
 576: 62,
 577: 0,
 586: 0,
 597: 73,
 598: 72,
 599: 1

In [6]:
sfr['segment'] = [r[s] for s in sfr.segment]
sfr['outseg'] = [r[s] for s in sfr.outseg]
sfr[['segment', 'outseg']]

,segment,outseg
0,1,6
1,1,6
2,1,6
3,1,6
4,1,6
5,7,11
6,7,11
7,7,11
8,7,11
9,7,11


In [8]:
sfr.sort_values(['segment', 'reach'], inplace=True)
# make sure reaches start with 1 in each segment
for s in np.unique(sfr.segment):
    df = sfr.loc[sfr.segment == s]
    sfr.loc[sfr.segment == s, 'reach'] = np.arange(1, len(df) + 1)

In [9]:
sfr

,comid,geometry,node,outseg,reach,reachID,segment
0,1815991,LINESTRING (695822.8050969027 5134496.31182678...,686248,6,1,45717,1
1,1815991,"LINESTRING (695822.6956740823 5134480.3, 69582...",687476,6,2,45718,1
2,1815991,"LINESTRING (695822.1749328185 5134404.1, 69582...",688704,6,3,45719,1
3,1815991,LINESTRING (695821.6541915546 5134327.89999999...,689932,6,4,45720,1
4,1815991,LINESTRING (695821.1334502909 5134251.69999999...,691160,6,5,45721,1
179,1815019,LINESTRING (695059.0911317427 5137912.08001123...,630978,9,1,32833,2
180,1815019,LINESTRING (695089.1000000001 5137951.50378540...,630979,9,2,32834,2
181,1815019,LINESTRING (695111.9588088206 5137985.49999999...,629751,9,3,32835,2
182,1815019,"LINESTRING (695089.1 5138009.498577582, 695088...",629750,9,4,32836,2
183,1815019,LINESTRING (695087.4756599031 5138061.69999999...,628522,9,5,32837,2


In [25]:
sfr.outseg.values

array([198, 198, 198, ..., 785, 785, 785])

In [10]:
start_cds = [(g.xy[0][0], g.xy[1][0]) for g in nl.geometry]
end_cds = [(g.xy[0][-1], g.xy[1][-1]) for g in nl.geometry]
sfr_start_cds = [(g.xy[0][0], g.xy[1][0]) for g in sfr.geometry]

In [11]:
max_elevs = get_values_at_points(dem, start_cds)
min_elevs = get_values_at_points(dem, end_cds)

In [12]:
nl = pd.DataFrame(nl.geometry)
nl['segment'] = np.arange(nsegstart, nsegstart+len(nl))
nl['elevMax'] = max_elevs
nl['elevMin'] = min_elevs
nl['upsegs'] = [[]] * len(nl)
nl['outseg'] = 0

In [125]:
# get index of nearest start to each end
def get_nearest(starts, ends):
    """Returns index of nearest start coordinate to each coordinate in ends.
    
    Parameters
    ----------
    starts : list of tuples
        Could be starting coordinates of each LineString.
    ends : list of tuples
        Could be ending coordinates of each LineString.
    """
    idx = index.Index()
    for i, start in enumerate(starts):
        idx.insert(i, start)
    return [list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
            else list(idx.nearest(end, 2))[1]
            for i, end in enumerate(ends)]


class lines:

    def __init__(self, shapefile, routing_tol=100):
        
        self.df = shp2df(shapefile)
        self.start_cds = [(g.xy[0][0], g.xy[1][0]) for g in self.df.geometry]
        self.end_cds = [(g.xy[0][-1], g.xy[1][-1]) for g in self.df.geometry]
        
        self.df['segment'] = np.arange(1, len(self.df) + 1)
        self.df['outsegs'] = 0
        self.df['upsegs'] = [[]] * len(self.df)
        
    def get_end_elevs_from_dem(self, dem):
        
        from GISio import get_values_at_points
        self.df['elevMax'] = get_values_at_points(dem, start_cds)
        self.df['elevMin'] = get_values_at_points(dem, end_cds)
        
    def route_lines_by_proximity(self):
        
        print 'routing new lines...'
        nearest_start = get_nearest(self.start_cds, self.end_cds)

        # record the preliminary seg. number of nearest start if within tol
        self.df['outseg'] = [self.df.segment[n] if Point(*start_cds[n]).distance(Point(*end_cds[i])) < tol 
                 else 0 for i, n in enumerate(nearest_start)]

        self.df['upsegs'] = [self.df.segment[self.df.outseg == s].tolist() for s in self.df.segment]
        
    def route_lines_to_sfr(self, sfrlinework, route2reach1=True, trim_buffer=20):
        """Route the linework in the lines class to an existing
        set of lines representing an SFR package.
        
        Parameters
        ----------
        sfrlinework : str (shapefile path) or dataframe
            Contains linework representing SFR package (e.g. already broken by grid)
        route2reach1 : boolean
            If true, linework is routed to closest starting coordinate of an SFR segment.
            Otherwise, routing is to closest starting coordinate of an SFR reach 
            (existing SFR segment in that location will have to be subdivided).
        """
            
        if not isinstance(sfrlinework, pd.DataFrame):
            self.sfr = shp2df(sfrlinework)
        else:
            self.sfr = sfrlinework.copy()
            
        if route2reach1:
            segments = self.sfr.ix[self.sfr.reach == 1, 'segment'].tolist()
            geoms = self.sfr.ix[self.sfr.reach == 1, 'geometry'].tolist()
        else:
            segments = self.sfr.segment.tolist()
            geoms = [g for g in sfr.geometry]
        
        # update segment numbering so that it starts after highest seg in sfr dataset
        if len(set(self.df.segment).intersection(self.sfr.segment)) != 0:
            maxseg = self.sfr.segment.max()
            self.df['segment'] += maxseg
            self.df['outseg'] += maxseg
            self.df['upsegs'] = [[u + maxseg for u in us] for us in lns.df.upsegs.tolist()]
            
        sfr_start_cds = [(g.xy[0][0], g.xy[1][0]) for g in geoms]
            
        print 'routing new lines to SFR...'

        is_outlet = self.df.outseg.values == 0
        new_lines_outlet_cds = map(tuple, np.array(self.end_cds)[is_outlet])

        # get index of nearest start to each end
        
        nearest_sfr = get_nearest(sfr_start_cds, new_lines_outlet_cds)

        # record the preliminary seg. number of nearest start if within tol
        self.df.loc[is_outlet, 'outseg'] = [segments[n] 
                                       if Point(*sfr_start_cds[n]).distance(\
                                          Point(*new_lines_outlet_cds[i])) < tol 
                                       else 0 for i, n in enumerate(nearest_sfr)]
        if not route2reach1:
            reaches = self.sfr.reach.tolist()
            self.df.loc[is_outlet, 'outreach'] = [reaches[n] 
                                       if Point(*sfr_start_cds[n]).distance(\
                                          Point(*new_lines_outlet_cds[i])) < tol 
                                       else 0 for i, n in enumerate(nearest_sfr)]
    
        # trim ends of new lines and then connect them to sfr start coords
        '''diffs = [g.difference(Point(g.coords[-1]).buffer(trim_buffer)) for g in self.df.geometry[is_outlet]]
        newgeoms = [LineString(list(g) + [sfr_start_cds[i]]) if g.length > 0 else 
                    LineString([g.coords[0], sfr_start_cds[i]]) for i, g in enumerate(diffs)]
        
        self.df.loc[is_outlet, 'geometry'] = newgeoms'''
        
        #return sfr_start_cds, new_lines_outlet_cds

In [132]:
trim_buffer=20
lns = lines(newlines)
lns.get_end_elevs_from_dem(dem)
lns.route_lines_by_proximity()
lns.route_lines_to_sfr(sfr, route2reach1=False)
outletlinegeoms = [g for g in lns.df.geometry[is_outlet]]
diffs = [g.difference(Point(g.coords[-1]).buffer(trim_buffer)) for g in outletlinegeoms]
newgeoms = [LineString(list(g.coords) + [sfr_start_cds[i]]) if g.length > 0 else 
            LineString([outletlinegeoms[i].coords[0], sfr_start_cds[i]]) for i, g in enumerate(diffs)]

lns.df.loc[is_outlet, 'geometry'] = newgeoms
        


reading ../Examples/data/added_lines.shp...
--> building dataframe... (may take a while for large shapefiles)
routing new lines...
routing new lines to SFR...


IndexError: list index out of range

In [ ]:
df2shp(lns.df, 'temp/junk.shp', epsg=26715)

In [153]:
reach = np.array([np.arange(1, 6).tolist() * 2])
segment = np.array([[1]*5 + [2]*5])
segment, reach

(array([[1, 1, 1, 1, 1, 2, 2, 2, 2, 2]]),
 array([[1, 2, 3, 4, 5, 1, 2, 3, 4, 5]]))

In [135]:
len(diffs)

55

In [88]:
sfr.segment

0        733
1        733
2        733
3        733
4        733
5        733
6        733
7        733
8        733
9        733
10       733
11       733
12       733
13       733
14       733
15       733
16       733
17       733
18       733
19       733
20       733
21       737
22       737
23       737
24       737
25       737
26       737
27       737
28       737
29       737
        ... 
1565    1139
1566    1139
1567    1139
1568    1139
1569    1139
1570    1139
1571    1139
1572    1139
1573    1139
1574    1139
1575    1139
1576    1139
1577    1139
1578    1139
1579    1139
1580    1139
1581    1139
1582    1139
1583    1139
1584    1139
1585    1139
1586    1139
1587    1228
1588    1228
1589    1228
1590    1228
1591    1228
1592    1228
1593    1228
1594    1228
Name: segment, dtype: int64

In [142]:
# get index of nearest start to each end
def get_nearest(starts, ends):
    idx = index.Index()
    for i, start in enumerate(starts):
        idx.insert(i, start)
    return [list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
            else list(idx.nearest(end, 2))[1]
            for i, end in enumerate(ends)]

#def route_lines_by_proximity(start_crds, end_crds)
print 'routing new lines...'
nearest_start = get_nearest(start_cds, end_cds)

# record the preliminary seg. number of nearest start if within tol
nl['outseg'] = [nl.segment[n] if Point(*start_cds[n]).distance(Point(*end_cds[i])) < tol 
                 else 0 for i, n in enumerate(nearest_start)]

nl['upsegs'] = [nl.segment[nl.outseg == s].tolist() for s in nl.segment]

routing new lines...


AttributeError: 'DataFrame' object has no attribute 'segment'

In [15]:
print 'routing new lines to SFR...'

is_outlet = nl.outseg.values == 0
new_lines_outlet_cds = map(tuple, np.array(end_cds)[is_outlet])

# get index of nearest start to each end
nearest_sfr = get_nearest(sfr_start_cds, new_lines_outlet_cds)

# record the preliminary seg. number of nearest start if within tol
nl.loc[is_outlet, 'outseg'] = [sfr.segment[n] 
                               if Point(*sfr_start_cds[n]).distance(\
                                  Point(*new_lines_outlet_cds[i])) < tol 
                               else 0 for i, n in enumerate(nearest_sfr)]

routing new lines to SFR...


In [17]:
df2shp(nl, 'temp/junk.shp', epsg=26715)

writing temp/junk.shp...


In [ ]:
[Point(*sfr_start_cds[n]).distance(\
Point(*new_lines_outlet_cds[i])) 
for i, n in enumerate(nearest_sfr)]

In [ ]:
idx = index.Index()
for i, start in enumerate(start_cds):
    idx.insert(i, start)
[list(idx.nearest(end, 2))[0] if list(idx.nearest(end, 1))[0] != i
        else list(idx.nearest(end, 2))[1]
        for i, end in enumerate(end_cds)]

In [ ]:
[i for i, end in enumerate(end_cds)]